### Processor
* VOC_duration : 3sec
* Wash_Out : 8min

### Sensors
+ TGS : TGS2600, TGS2602, TGS2603, TGS2610, TGS2620
+ MQ : MQ2, MQ3, MQ4, MQ5, MQ6, MQ7, MQ8, MQ9, MQ135

### VOC
1. C.albicans
2. C.glabrata
3. E.coli
4. K.pneumoniae
5. S.aureus
6. S.epidermidis

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pymysql
import matplotlib.pyplot as plt
# import config
import datetime, time
import os
from pathlib import Path
from tqdm import tqdm
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
plot_style = plt.style.available
# pd.options.display.float_format = '{:.5f}'.format
# pd.set_option('display.max_row', 500)
pd.set_option('display.max_rows', 500)

In [3]:
# plt.style.use('ggplot')
# plt.style.use('default')
# plt.style.use('fivethirtyeight')
plt.style.use('seaborn-paper')
# plt.style.use('seaborn-muted')

In [4]:
pd.set_option('display.expand_frame_repr', False)
np.random.seed(42)

In [5]:
path = os.getcwd()
data_dir = './data/Microbe_data/'
data_dir

'./data/Microbe_data/'

In [6]:
file_list = os.listdir(data_dir)
sensor_files = sorted([file for file in file_list if "sensor" in file])
# sensor_files
valve_files = sorted([file for file in file_list if "valve" in file])
# valve_files

In [7]:
sensor_valve_pair = [(sensor, valve) for sensor, valve in zip(sensor_files, valve_files)]
sensor_valve_pair

[('C.albicans_sensor_1 (1).csv', 'C.albicans_valve_1 (1).csv'),
 ('C.albicans_sensor_2 (1).csv', 'C.albicans_valve_2 (1).csv'),
 ('C.glabrata_sensor_1 (1).csv', 'C.glabrata_valve_1 (1).csv'),
 ('C.glabrata_sensor_2 (1).csv', 'C.glabrata_valve_2 (1).csv'),
 ('C.para_sensor_1 (1).csv', 'C.para_valve_1 (1).csv'),
 ('C.para_sensor_2 (1).csv', 'C.para_valve_2 (1).csv'),
 ('C.para_sensor_3 (1).csv', 'C.para_valve_3 (1).csv'),
 ('E.coli_sensor_1 (1).csv', 'E.coli_valve_1 (1).csv'),
 ('E.coli_sensor_2 (1).csv', 'E.coli_valve_2 (1).csv'),
 ('E.coli_sensor_3 (1).csv', 'E.coli_valve_3 (1).csv'),
 ('K.pneumoniae_sensor_1 (1).csv', 'K.pneumoniae_valve_1 (1).csv'),
 ('K.pneumoniae_sensor_2 (1).csv', 'K.pneumoniae_valve_2 (1).csv'),
 ('S.aureus_sensor_1 (1).csv', 'S.aureus_valve_1 (1).csv'),
 ('S.aureus_sensor_2 (1).csv', 'S.aureus_valve_2 (1).csv'),
 ('S.aureus_sensor_3 (1).csv', 'S.aureus_valve_3 (1).csv'),
 ('S.epidermidis_sensor_1 (1).csv', 'S.epidermidis_valve_1 (1).csv'),
 ('S.epidermidis_senso

In [8]:
PROJECT_DIR = Path('.')
RESULT_DIR = Path(PROJECT_DIR, 'result')
print(RESULT_DIR)

result


In [9]:
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [20]:
total_cycles = []
for sensor_file, valve_file in tqdm(sensor_valve_pair[:]):
    df_sensor = pd.read_csv(f'{data_dir}{sensor_file}')
    df_sensor['Time'] = pd.to_datetime(df_sensor['Time'])
    df_sensor.set_index('Time', inplace=True)
    df_sensor = df_sensor.drop(['Time.1'], axis=1).replace(0, np.nan)
    
    df_valve = pd.read_csv(f'{data_dir}{valve_file}')
    df_valve['Time'] = pd.to_datetime(df_valve['Time'])
    df_valve.set_index('Time', inplace=True)
    df_valve = df_valve.drop(['Time.1', 'd_num'], axis=1).replace(0, np.nan)
    
    sensors = ['TGS2600', 'TGS2602', 'TGS2603', 'TGS2620', 'TGS2610', 'TGS822', 
               'MQ2', 'MQ3', 'MQ4', 'MQ5', 'MQ6', 'MQ7', 'MQ8', 'MQ9', 'MQ135']
#     sample = sensor_file.split(' ')[0].split('_')[0]
    sample = sensor_file.split(' ')[0]
    
    path = Path(RESULT_DIR, 'figure', '8min', sample)
    createDirectory(path)
    
    cycles = []
    for i in range(len(df_valve)-1):
        cycle_temp = {}
        if df_valve.iloc[i]['VOC'] == df_valve.iloc[i+1]['VOC']:
            cycle_temp['sample'] = sample
            cycle_temp['cycle'] = int(i/2)
            cycle_temp['VOC'] = df_valve.iloc[i]['VOC'].astype(int)
            cycle_temp['cycle_start'] = df_valve.index[i] - pd.Timedelta(seconds=4)
            cycle_temp['valve_open'] = df_valve.index[i]
            cycle_temp['valve_close'] = df_valve.index[i+1]
            cycle_temp['cycle_end'] = df_valve.index[i]+ pd.Timedelta(minutes=8)

            cycles.append(cycle_temp)
            total_cycles.append(cycle_temp)
            
#     print(cycles)


######################################################
# Figure generation (every single cycle)
######################################################
            
#     print(f'Cycle 총수: {len(cycles)}')
        
    for idx, cycle in enumerate(cycles[:]):
        cycle_cond = (cycle['cycle_start'] < df_sensor.index)  & (df_sensor.index < cycle['cycle_end'])
    
        title = str(sample) + ' VOC' + str(int(cycle['VOC']))
        title = title   + ' Cycle_' + str(idx).zfill(3) + '-' + str(len(cycles))

        plt.figure(figsize=(9, 6))
        try:
            for sensor in sensors:
                df_sensor[cycle_cond][sensor].dropna().plot()
                plt.title(title, fontsize=14, loc="left")
                plt.legend(loc='upper left', bbox_to_anchor=(0.95, 1))
                plt.axvspan(cycle['valve_open'], cycle['valve_close'], color="gray", alpha=0.01)
        except IndexError as e:
            print(e)
            print()
        plt.savefig(str(path)+'/'+title+'.png')
        plt.show()

100% 18/18 [00:09<00:00,  1.85it/s]
